In [2]:
import pandas as pd
import requests
from tqdm import tqdm
import time
from datetime import datetime
import numpy as np
import csv
import pickle
import urllib

In [3]:
# api key
headers = {
  "x-nxopen-api-key": "test_b7a2709bb4fe7572494ad6f8f3ce6f4a9a08ece44f72cc3275a9c0fe0bc1031c1007179e9a522fc8756d42cac9e0b876",
}


In [4]:
# 현재 날짜
now = datetime.now()
now_date = now.date()

In [5]:
# 직업명 리스트
job_list = ['warrior','warrior_hero','warrior_paladin','warrior_darkkinght','magician','magician_fire','magician_ice','magician_bishop','archer','archer_bowmaster','archer_marksman','archer_pathfinder','thief','thief_nightlord','thief_shadower','thief_blademaster','pirate','pirate_viper','pirate_captain','pirate_cannon','cygnus','cygnus_soul','cygnus_flame','cygnus_wind','cygnus_night','cygnus_striker','cygnus_mihile','aran','evan','resistance','resistance_battle','resistance_wild','resistance_mechanic','resistance_slayer','resistance_avenger','resistance_xenon','resistance_blaster','mercedes','phantom','luminous','kaiser','angelic','zero','shade','kinesis','cadena','illium','ark','hoyoung','adele','kain','lara','khali']

In [6]:
# 전체 직업 request URL(초보자, 기본직업 등 제외) / (now_date 변수 통해 현재 날짜 삽입)
url_list = [
    f'https://open.api.nexon.com/maplestory/v1/ranking/overall?date={now_date}&class=%EC%A0%84%EC%82%AC-%EC%A0%84%EC%B2%B4%20%EC%A0%84%EC%A7%81',
    f'https://open.api.nexon.com/maplestory/v1/ranking/overall?date={now_date}&class=%EC%A0%84%EC%82%AC-%ED%9E%88%EC%96%B4%EB%A1%9C',
    f'https://open.api.nexon.com/maplestory/v1/ranking/overall?date={now_date}&class=%EC%A0%84%EC%82%AC-%ED%8C%94%EB%9D%BC%EB%94%98',
    f'https://open.api.nexon.com/maplestory/v1/ranking/overall?date={now_date}&class=%EC%A0%84%EC%82%AC-%EB%8B%A4%ED%81%AC%EB%82%98%EC%9D%B4%ED%8A%B8',
    f'https://open.api.nexon.com/maplestory/v1/ranking/overall?date={now_date}&class=%EB%A7%88%EB%B2%95%EC%82%AC-%EC%A0%84%EC%B2%B4%20%EC%A0%84%EC%A7%81',
    f'https://open.api.nexon.com/maplestory/v1/ranking/overall?date={now_date}&class=%EB%A7%88%EB%B2%95%EC%82%AC-%EC%95%84%ED%81%AC%EB%A9%94%EC%9D%B4%EC%A7%80%28%EB%B6%88%2C%EB%8F%85%29',
    f'https://open.api.nexon.com/maplestory/v1/ranking/overall?date={now_date}&class=%EB%A7%88%EB%B2%95%EC%82%AC-%EC%95%84%ED%81%AC%EB%A9%94%EC%9D%B4%EC%A7%80%28%EC%8D%AC%2C%EC%BD%9C%29',
    f'https://open.api.nexon.com/maplestory/v1/ranking/overall?date={now_date}&class=%EB%A7%88%EB%B2%95%EC%82%AC-%EB%B9%84%EC%88%8D',
    f'https://open.api.nexon.com/maplestory/v1/ranking/overall?date={now_date}&class=%EA%B6%81%EC%88%98-%EC%A0%84%EC%B2%B4%20%EC%A0%84%EC%A7%81',
    f'https://open.api.nexon.com/maplestory/v1/ranking/overall?date={now_date}&class=%EA%B6%81%EC%88%98-%EB%B3%B4%EC%9A%B0%EB%A7%88%EC%8A%A4%ED%84%B0',
    f'https://open.api.nexon.com/maplestory/v1/ranking/overall?date={now_date}&class=%EA%B6%81%EC%88%98-%EC%8B%A0%EA%B6%81',
    f'https://open.api.nexon.com/maplestory/v1/ranking/overall?date={now_date}&class=%EA%B6%81%EC%88%98-%ED%8C%A8%EC%8A%A4%ED%8C%8C%EC%9D%B8%EB%8D%94',
    f'https://open.api.nexon.com/maplestory/v1/ranking/overall?date={now_date}&class=%EB%8F%84%EC%A0%81-%EC%A0%84%EC%B2%B4%20%EC%A0%84%EC%A7%81',
    f'https://open.api.nexon.com/maplestory/v1/ranking/overall?date={now_date}&class=%EB%8F%84%EC%A0%81-%EB%82%98%EC%9D%B4%ED%8A%B8%EB%A1%9C%EB%93%9C',
    f'https://open.api.nexon.com/maplestory/v1/ranking/overall?date={now_date}&class=%EB%8F%84%EC%A0%81-%EC%84%80%EB%8F%84%EC%96%B4',
    f'https://open.api.nexon.com/maplestory/v1/ranking/overall?date={now_date}&class=%EB%8F%84%EC%A0%81-%EB%93%80%EC%96%BC%EB%B8%94%EB%A0%88%EC%9D%B4%EB%8D%94',
    f'https://open.api.nexon.com/maplestory/v1/ranking/overall?date={now_date}&class=%ED%95%B4%EC%A0%81-%EC%A0%84%EC%B2%B4%20%EC%A0%84%EC%A7%81',
    f'https://open.api.nexon.com/maplestory/v1/ranking/overall?date={now_date}&class=%ED%95%B4%EC%A0%81-%EB%B0%94%EC%9D%B4%ED%8D%BC',
    f'https://open.api.nexon.com/maplestory/v1/ranking/overall?date={now_date}&class=%ED%95%B4%EC%A0%81-%EC%BA%A1%ED%8B%B4',
    f'https://open.api.nexon.com/maplestory/v1/ranking/overall?date={now_date}&class=%ED%95%B4%EC%A0%81-%EC%BA%90%EB%85%BC%EB%A7%88%EC%8A%A4%ED%84%B0',
    f'https://open.api.nexon.com/maplestory/v1/ranking/overall?date={now_date}&class=%EA%B8%B0%EC%82%AC%EB%8B%A8-%EC%A0%84%EC%B2%B4%20%EC%A0%84%EC%A7%81',
    f'https://open.api.nexon.com/maplestory/v1/ranking/overall?date={now_date}&class=%EA%B8%B0%EC%82%AC%EB%8B%A8-%EC%86%8C%EC%9A%B8%EB%A7%88%EC%8A%A4%ED%84%B0',
    f'https://open.api.nexon.com/maplestory/v1/ranking/overall?date={now_date}&class=%EA%B8%B0%EC%82%AC%EB%8B%A8-%ED%94%8C%EB%A0%88%EC%9E%84%EC%9C%84%EC%9E%90%EB%93%9C',
    f'https://open.api.nexon.com/maplestory/v1/ranking/overall?date={now_date}&class=%EA%B8%B0%EC%82%AC%EB%8B%A8-%EC%9C%88%EB%93%9C%EB%B8%8C%EB%A0%88%EC%9D%B4%EC%BB%A4',
    f'https://open.api.nexon.com/maplestory/v1/ranking/overall?date={now_date}&class=%EA%B8%B0%EC%82%AC%EB%8B%A8-%EB%82%98%EC%9D%B4%ED%8A%B8%EC%9B%8C%EC%BB%A4',
    f'https://open.api.nexon.com/maplestory/v1/ranking/overall?date={now_date}&class=%EA%B8%B0%EC%82%AC%EB%8B%A8-%EC%8A%A4%ED%8A%B8%EB%9D%BC%EC%9D%B4%EC%BB%A4',
    f'https://open.api.nexon.com/maplestory/v1/ranking/overall?date={now_date}&class=%EA%B8%B0%EC%82%AC%EB%8B%A8-%EB%AF%B8%ED%95%98%EC%9D%BC',
    f'https://open.api.nexon.com/maplestory/v1/ranking/overall?date={now_date}&class=%EC%95%84%EB%9E%80-%EC%A0%84%EC%B2%B4%20%EC%A0%84%EC%A7%81',
    f'https://open.api.nexon.com/maplestory/v1/ranking/overall?date={now_date}&class=%EC%97%90%EB%B0%98-%EC%A0%84%EC%B2%B4%20%EC%A0%84%EC%A7%81',
    f'https://open.api.nexon.com/maplestory/v1/ranking/overall?date={now_date}&class=%EB%A0%88%EC%A7%80%EC%8A%A4%ED%83%95%EC%8A%A4-%EC%A0%84%EC%B2%B4%20%EC%A0%84%EC%A7%81',
    f'https://open.api.nexon.com/maplestory/v1/ranking/overall?date={now_date}&class=%EB%A0%88%EC%A7%80%EC%8A%A4%ED%83%95%EC%8A%A4-%EB%B0%B0%ED%8B%80%EB%A9%94%EC%9D%B4%EC%A7%80',
    f'https://open.api.nexon.com/maplestory/v1/ranking/overall?date={now_date}&class=%EB%A0%88%EC%A7%80%EC%8A%A4%ED%83%95%EC%8A%A4-%EC%99%80%EC%9D%BC%EB%93%9C%ED%97%8C%ED%84%B0',
    f'https://open.api.nexon.com/maplestory/v1/ranking/overall?date={now_date}&class=%EB%A0%88%EC%A7%80%EC%8A%A4%ED%83%95%EC%8A%A4-%EB%A9%94%EC%B9%B4%EB%8B%89',
    f'https://open.api.nexon.com/maplestory/v1/ranking/overall?date={now_date}&class=%EB%A0%88%EC%A7%80%EC%8A%A4%ED%83%95%EC%8A%A4-%EB%8D%B0%EB%AA%AC%EC%8A%AC%EB%A0%88%EC%9D%B4%EC%96%B4',
    f'https://open.api.nexon.com/maplestory/v1/ranking/overall?date={now_date}&class=%EB%A0%88%EC%A7%80%EC%8A%A4%ED%83%95%EC%8A%A4-%EB%8D%B0%EB%AA%AC%EC%96%B4%EB%B2%A4%EC%A0%B8',
    f'https://open.api.nexon.com/maplestory/v1/ranking/overall?date={now_date}&class=%EB%A0%88%EC%A7%80%EC%8A%A4%ED%83%95%EC%8A%A4-%EC%A0%9C%EB%85%BC',
    f'https://open.api.nexon.com/maplestory/v1/ranking/overall?date={now_date}&class=%EB%A0%88%EC%A7%80%EC%8A%A4%ED%83%95%EC%8A%A4-%EB%B8%94%EB%9E%98%EC%8A%A4%ED%84%B0',
    f'https://open.api.nexon.com/maplestory/v1/ranking/overall?date={now_date}&class=%EB%A9%94%EB%A5%B4%EC%84%B8%EB%8D%B0%EC%8A%A4-%EC%A0%84%EC%B2%B4%20%EC%A0%84%EC%A7%81',
    f'https://open.api.nexon.com/maplestory/v1/ranking/overall?date={now_date}&class=%ED%8C%AC%ED%85%80-%EC%A0%84%EC%B2%B4%20%EC%A0%84%EC%A7%81',
    f'https://open.api.nexon.com/maplestory/v1/ranking/overall?date={now_date}&class=%EB%A3%A8%EB%AF%B8%EB%84%88%EC%8A%A4-%EC%A0%84%EC%B2%B4%20%EC%A0%84%EC%A7%81',
    f'https://open.api.nexon.com/maplestory/v1/ranking/overall?date={now_date}&class=%EC%B9%B4%EC%9D%B4%EC%A0%80-%EC%A0%84%EC%B2%B4%20%EC%A0%84%EC%A7%81',
    f'https://open.api.nexon.com/maplestory/v1/ranking/overall?date={now_date}&class=%EC%97%94%EC%A0%A4%EB%A6%AD%EB%B2%84%EC%8A%A4%ED%84%B0-%EC%A0%84%EC%B2%B4%20%EC%A0%84%EC%A7%81',
    f'https://open.api.nexon.com/maplestory/v1/ranking/overall?date={now_date}&class=%EC%B4%88%EC%9B%94%EC%9E%90-%EC%A0%9C%EB%A1%9C',
    f'https://open.api.nexon.com/maplestory/v1/ranking/overall?date={now_date}&class=%EC%9D%80%EC%9B%94-%EC%A0%84%EC%B2%B4%20%EC%A0%84%EC%A7%81',
    f'https://open.api.nexon.com/maplestory/v1/ranking/overall?date={now_date}&class=%ED%94%84%EB%A0%8C%EC%A6%88%20%EC%9B%94%EB%93%9C-%ED%82%A4%EB%84%A4%EC%8B%9C%EC%8A%A4',
    f'https://open.api.nexon.com/maplestory/v1/ranking/overall?date={now_date}&class=%EC%B9%B4%EB%8D%B0%EB%82%98-%EC%A0%84%EC%B2%B4%20%EC%A0%84%EC%A7%81',
    f'https://open.api.nexon.com/maplestory/v1/ranking/overall?date={now_date}&class=%EC%9D%BC%EB%A6%AC%EC%9B%80-%EC%A0%84%EC%B2%B4%20%EC%A0%84%EC%A7%81',
    f'https://open.api.nexon.com/maplestory/v1/ranking/overall?date={now_date}&class=%EC%95%84%ED%81%AC-%EC%A0%84%EC%B2%B4%20%EC%A0%84%EC%A7%81',
    f'https://open.api.nexon.com/maplestory/v1/ranking/overall?date={now_date}&class=%ED%98%B8%EC%98%81-%EC%A0%84%EC%B2%B4%20%EC%A0%84%EC%A7%81',
    f'https://open.api.nexon.com/maplestory/v1/ranking/overall?date={now_date}&class=%EC%95%84%EB%8D%B8-%EC%A0%84%EC%B2%B4%20%EC%A0%84%EC%A7%81',
    f'https://open.api.nexon.com/maplestory/v1/ranking/overall?date={now_date}&class=%EC%B9%B4%EC%9D%B8-%EC%A0%84%EC%B2%B4%20%EC%A0%84%EC%A7%81',
    f'https://open.api.nexon.com/maplestory/v1/ranking/overall?date={now_date}&class=%EB%9D%BC%EB%9D%BC-%EC%A0%84%EC%B2%B4%20%EC%A0%84%EC%A7%81',
    f'https://open.api.nexon.com/maplestory/v1/ranking/overall?date={now_date}&class=%EC%B9%BC%EB%A6%AC-%EC%A0%84%EC%B2%B4%20%EC%A0%84%EC%A7%81'
]

In [127]:
# pickle 파일로 저장
for i in range(len(url_list)) :
    url = url_list[i]
    request_rank = requests.get(url, headers = headers)
    result_rank = request_rank.json()
    with open(f'{job_list[i]}_rank', 'wb') as fw:
        pickle.dump(result_rank, fw)

In [7]:
# pickle 파일 읽어오기
for i in range(len(job_list)) :
    with open(f'{job_list[i]}_rank', 'rb') as fr :
        globals()[f'{job_list[i]}_rank'] = pickle.load(fr)

In [13]:
# 캐릭터 식별자(ocid) 수집
for i in range(47,51) :
    name_list = []
    ocid_list = []
    for j in range(len(warrior_rank['ranking'])) :
        name_list.append(globals()[f'{job_list[i]}_rank']['ranking'][j]['character_name'])
    for k in name_list :
        try :
            ocid_url = 'https://open.api.nexon.com/maplestory/v1/id?character_name=' + urllib.parse.quote(k)
            request_ocid = requests.get(ocid_url, headers = headers)
            result_ocid = request_ocid.json()
            ocid_list.append(result_ocid['ocid'])
            time.sleep(0.3)
        except :
            time.sleep(0.3)
            pass
    with open(f'{job_list[i]}_ocid', 'w') as file :
        writer = csv.writer(file)
        writer.writerow(ocid_list)
        

In [9]:
name_list = []
ocid_list = []
for j in range(len(warrior_rank['ranking'])) :
    name_list.append(globals()[f'{job_list[18]}_rank']['ranking'][j]['character_name'])
for k in name_list :
    try :
        ocid_url = 'https://open.api.nexon.com/maplestory/v1/id?character_name=' + urllib.parse.quote(k)
        request_ocid = requests.get(ocid_url, headers = headers)
        result_ocid = request_ocid.json()
        ocid_list.append(result_ocid['ocid'])
        time.sleep(0.3)
    except :
        time.sleep(0.3)
        pass
with open(f'{job_list[18]}_ocid', 'w') as file :
    writer = csv.writer(file)
    writer.writerow(ocid_list)

In [23]:
# ark부터 다시 47
a = []
with open('hoyoung_ocid', newline='') as f:
    csv_reader = csv.reader(f)
    for row in csv_reader :
        a.append(row)
len(a[0])

0

In [25]:
job_list[47:51]

['ark', 'hoyoung', 'adele', 'kain']